<a href="https://colab.research.google.com/github/soufianeze/MachineLearning2/blob/main/MLFLOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install mlflow 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.8 MB 701 kB/s 
     |████████████████████████████████| 146 kB 54.5 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 596 kB 30.6 MB/s 
     |████████████████████████████████| 181 kB 32.4 MB/s 
     |████████████████████████████████| 209 kB 41.0 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.16.7-py3-none-any.whl size=119058 sha256=ed8e51c759fa33c90ddb7373e572a928d13fd8e5933cece43e71e5f1aca34662
  Stored in directory: /root/.cache/pip/wheels/4c/52/86/223563d0c8a3c4f25fb2bb9efab5d67654bc929a2182a71630
Successfully built databricks-cli
  Attempting uninstall

In [4]:
!mlflow --version

mlflow, version 1.26.1


In [9]:
import mlflow
import mlflow.sklearn
mlflow.set_experiment('LearnML-Demo')

<Experiment: artifact_location='file:///content/mlruns/1', experiment_id='1', lifecycle_stage='active', name='LearnML-Demo', tags={}>

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
import sys
import os

In [10]:
def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

In [12]:
! wget https://raw.githubusercontent.com/soufianeze/MachineLearning2/main/winequality-red.csv

--2022-06-12 21:50:38--  https://raw.githubusercontent.com/soufianeze/MachineLearning2/main/winequality-red.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85797 (84K) [text/plain]
Saving to: ‘winequality-red.csv’

winequality-red.csv 100%[===================>]  83.79K  --.-KB/s    in 0.01s   

2022-06-12 21:50:38 (5.62 MB/s) - ‘winequality-red.csv’ saved [85797/85797]



In [13]:
data = pd.read_csv("winequality-red.csv",delimiter=";")

In [14]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [20]:
# Read the wine-quality csv file


def train_model(alpha,l1_ratio):
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [21]:
train_model(0.5,0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.752118364932758
  MAE: 0.602616521305871
  R2: 0.13764696083063332


In [22]:
train_model(0.2,0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7139503266547771
  MAE: 0.5628614469932045
  R2: 0.20818638016996238


In [23]:
train_model(0.1,0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.6563550311748853
  MAE: 0.5179824504345802
  R2: 0.3156374833767853


In [24]:
train_model(0.4,0.1)

Elasticnet model (alpha=0.400000, l1_ratio=0.100000):
  RMSE: 0.6768348556596335
  MAE: 0.553090938032629
  R2: 0.24949963657307184


In [25]:
train_model(0.1,0.4)

Elasticnet model (alpha=0.100000, l1_ratio=0.400000):
  RMSE: 0.688112645699284
  MAE: 0.5406528597804315
  R2: 0.2898668768756053


In [18]:
pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=e158fadb37ced1b4d2b8ef57808bdc18ca6890608f718b260b2a7252ac0b3c83
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [26]:
from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://4545-34-73-215-151.ngrok.io
